In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
X_train = train_data.drop("Detected", axis=1)

In [8]:
y_train = train_data.Detected

In [9]:
X_valid = valid_data.drop("Detected", axis=1)

In [10]:
y_valid = valid_data.Detected

In [11]:
with open(dPath / "rf_exp_04_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [12]:
X_train = X_train[names] 

In [13]:
X_valid = X_valid[names]

In [14]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [15]:
from imblearn.over_sampling import ADASYN

In [16]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [17]:
%time X_train, y_train = sm.fit_resample(X_train, y_train)

Wall time: 54.3 s


In [18]:
from catboost import CatBoostClassifier, Pool, cv

In [20]:
CatBoostClassifier?

In [20]:
tqdm?

In [24]:
models = []
np.random.seed(42)

for i in tqdm(range(50)):
    
    cb = CatBoostClassifier(
    custom_loss=['AUC'],
    random_seed=np.random.randint(2 ** 10),
    cat_features=[1],
    iterations=1000,
    logging_level='Silent')

    cb.fit(X_train, y_train)
    models.append(cb.copy())

C:\Users\Alireza\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [25]:
with open(dPath / "cb_exp_04_stacking.pkl", 'wb') as filename:
    pickle.dump(models,filename)

In [26]:
predictions = []
for _model in tqdm(models):
    predictions.append(_model.predict_proba(X_valid)[:,1])
    
predictions = np.vstack(predictions).T

C:\Users\Alireza\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [27]:
prediction = np.hstack([predictions]).mean(axis=1)

In [28]:
prediction > 0.5

array([False, False, False, ..., False, False,  True])

In [29]:
from sklearn.metrics import classification_report

In [30]:
print(classification_report(y_valid, prediction >=0.5))

              precision    recall  f1-score   support

       False       0.40      0.41      0.40     16657
        True       0.81      0.80      0.80     50839

    accuracy                           0.70     67496
   macro avg       0.60      0.60      0.60     67496
weighted avg       0.70      0.70      0.70     67496



In [31]:
from sklearn import metrics

In [32]:
fpr, tpr, thresholds = metrics.roc_curve(y_valid, prediction)

In [33]:
metrics.auc(fpr, tpr)

0.665678015001686

In [74]:
with open(dPath / "cb_exp_03_stacking.pkl", 'rb') as filename:
    cbs = pickle.load(filename)

In [35]:
np.cumsum([cbs[i].predict_proba(X_valid)[0,1] for i in range(20)]) / np.arange(1,21)

array([0.0134298 , 0.0330026 , 0.04979718, 0.04555388, 0.13790174,
       0.17681601, 0.16581802, 0.15807399, 0.15077116, 0.14088243,
       0.13770488, 0.12954822, 0.1418545 , 0.15157425, 0.1453652 ,
       0.13689089, 0.1312241 , 0.12746202, 0.12995397, 0.13220325])